In [22]:
import pandas as pd

off_train = pd.read_csv('data/ccf_offline_stage1_train.csv',parse_dates=["Date"],header=0)
off_train.columns = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']

off_test = pd.read_csv('data/ccf_offline_stage1_test_revised.csv',header=0)
off_test.columns = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received']

on_train = pd.read_csv('data/ccf_online_stage1_train.csv',parse_dates=["Date"],header=0)
on_train.columns = ['user_id','merchant_id','action','coupon_id','discount_rate','date_received','date']

# 【dataset3】20160701~20160731 (113640)   【feature3】20160315~20160630  （测试集）
dataset3 = off_test.dropna(how='any')
feature3 = off_train[((off_train.date >= '20160315') & (off_train.date <= '20160630'))
                   | ((off_train.date == 'null') & (off_train.date_received >= '20160315') 
                      &  (off_train.date_received <= '20160630'))
                   ].dropna(how='any')
# 【dataset2】20160515~20160615 (258446)   【feature2】20160201~20160514  （训练集2）
dataset2 = off_train[((off_train.date_received >= '20160515') & (off_train.date_received <= '20160615'))].dropna(how='any')
feature2 = off_train[((off_train.date >= '20160201') & (off_train.date <= '20160514'))
                   | ((off_train.date == 'null') & (off_train.date_received >= '20160201') 
                      &  (off_train.date_received <= '20160514'))
                   ].dropna(how='any')
# 【dataset1】20160414~20160514 (138303)   【feature1】20160101~20160413  （训练集1）
dataset1 = off_train[((off_train.date_received >= '20160414') & (off_train.date_received <= '20160514'))].dropna(how='any')
feature1 = off_train[((off_train.date >= '20160101') & (off_train.date <= '20160413'))
                   | ((off_train.date == 'null') & (off_train.date_received >= '20160101') 
                      &  (off_train.date_received <= '20160413'))
                   ].dropna(how='any')

In [23]:
from datetime import date
import numpy as np

In [24]:
def get_discount_man(s):
    s = s.split(':')
    if len(s) == 1:
        return 'null'
    else:
        return s[0]
def get_discount_jian(s):
    s = s.split(':')
    if len(s) == 1:
        return 'null'
    else:
        return s[1]
def is_man_jian(s):
    s = s.split(':')
    if len(s) == 1:
        return 0
    else:
        return 1
def calc_discount_rate(s):
    s = s.split(':')
    if len(s) == 1: #直接折扣
        return float(s[0])
    else: #满X减Y
        return 1-float(s[1])/float(s[0])

In [25]:
def couponRelationFeature(dataset , file_name,flag):
    if flag:
        coupon = dataset[['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']]
    else :
        coupon = dataset[['user_id','merchant_id','coupon_id','discount_rate','distance','date_received']]
    #day_of_week # 显示时间是第几周
    coupon['day_of_week'] = coupon.date_received.astype('str').apply(
            lambda x: date(int(x[0:4]), int(x[4:6]), int(x[6:8])).weekday() + 1)
    coupon.ix[:5]
    #day_of_month # 显示时间是某月的某日
    coupon['day_of_month'] = coupon.date_received.astype('str').apply(
            lambda x: int(x[6:8]))
    coupon.ix[:5]
    #days_distance# 优惠券领取日期到截止日之间的间隔天数
    coupon['days_distance'] = coupon.date_received.astype('str').apply(
            lambda x: (date(int(x[0:4]), int(x[4:6]), int(x[6:8])) - date(2016, 6, 30)).days)
    coupon.ix[:5]
    # discount_man # 显示满了多少钱后开始减
    coupon['discount_man'] = coupon.discount_rate.apply(get_discount_man)
    coupon.ix[:5]
    # discount_jian # 显示满减的减少的钱
    coupon['discount_jian'] = coupon.discount_rate.apply(get_discount_jian)
    coupon.ix[:5]
    #  is_man_jian # 返回优惠券是否是满减券
    coupon['is_man_jian'] = coupon.discount_rate.apply(is_man_jian)
    coupon.ix[:5]
    
    # discount_rate # 优惠券折扣率
    coupon['discount_rate'] = coupon.discount_rate.apply(calc_discount_rate)
    coupon.ix[:5]
    
    t = coupon[['coupon_id']]
    t['coupon_count'] = 1
    t = t.groupby('coupon_id').agg('sum').reset_index()
    coupon = pd.merge(coupon, t, on=['coupon_id'], how='left')
    coupon.to_csv(file_name, index=None)
    return coupon

In [26]:
coupon3_feature = couponRelationFeature(dataset3 , 'data/coupon3_feature.csv',False)
coupon3_feature.ix[:5]

C:\Users\xhb_1\Anaconda3\lib\site-packages\ipykernel\__main__.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,day_of_week,day_of_month,days_distance,discount_man,discount_jian,is_man_jian,coupon_count
0,4129537,450,9983,0.833333,1,20160712,2,12,12,30,5,1,11586
1,6949378,1300,3429,0.833333,null,20160706,3,6,6,30,5,1,4887
2,2166529,7113,6928,0.900000,5,20160727,3,27,27,200,20,1,205
3,2166529,7113,1808,0.900000,5,20160727,3,27,27,100,10,1,205
4,6172162,7605,6500,0.966667,2,20160708,5,8,8,30,1,1,132
5,4005121,450,9983,0.833333,0,20160706,3,6,6,30,5,1,11586


In [27]:
coupon2_feature = couponRelationFeature(dataset2 , 'data/coupon2_feature.csv',True)
coupon2_feature.ix[:5]

C:\Users\xhb_1\Anaconda3\lib\site-packages\ipykernel\__main__.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date,day_of_week,day_of_month,days_distance,discount_man,discount_jian,is_man_jian,coupon_count
0,1439408,4663,11002,0.866667,1,20160528,null,6,28,-33,150,20,1,7730
1,1439408,2632,8591,0.950000,0,20160613,null,1,13,-17,20,1,1,5
2,1439408,2632,8591,0.950000,0,20160516,20160613,1,16,-45,20,1,1,5
3,2029232,450,1532,0.833333,0,20160530,null,1,30,-31,30,5,1,11728
4,2029232,6459,12737,0.950000,0,20160519,null,4,19,-42,20,1,1,16
5,2747744,6901,1097,0.800000,null,20160606,null,1,6,-24,50,10,1,483


In [28]:
coupon1_feature = couponRelationFeature(dataset1 , 'data/coupon1_feature.csv',True)
coupon1_feature.ix[:5]

C:\Users\xhb_1\Anaconda3\lib\site-packages\ipykernel\__main__.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date,day_of_week,day_of_month,days_distance,discount_man,discount_jian,is_man_jian,coupon_count
0,1832624,3381,7610,0.900000,0,20160429,null,5,29,-62,200,20,1,22281
1,163606,1569,5054,0.850000,10,20160421,null,4,21,-70,200,30,1,11893
2,4061024,3381,7610,0.900000,10,20160426,null,2,26,-65,200,20,1,22281
3,106443,450,3732,0.833333,null,20160429,null,5,29,-62,30,5,1,8845
4,114747,1569,5054,0.850000,9,20160426,null,2,26,-65,200,30,1,11893
5,6038208,1569,5054,0.850000,0,20160417,null,7,17,-74,200,30,1,11893
